In [94]:
# Importing libraries
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
import os

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support

from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor

Load  Dataset

In [95]:
# Load datasets

data1 = pd.read_excel(r"C:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\Datasets\case_study1.xlsx")
data2 = pd.read_excel(r"C:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\Datasets\case_study2.xlsx")


In [96]:
df1 = data1.copy()
df2 = data2.copy()

In [97]:
df1.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [98]:
df2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [99]:
df1.shape

(51336, 26)

Removing Null Values


In [100]:
df2.shape

(51336, 62)

In [101]:
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

In [102]:
df1.shape

(51296, 26)

In [103]:
# Removing null values

columns_to_be_removed = []
for i in df2.columns:
    # shape[0] is for rows
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed.append(i)


In [104]:
df2.head()

,PROSPECTID,time_since_recent_payment,time_since_first_deliquency,time_since_recent_deliquency,num_times_delinquent,max_delinquency_level,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,...,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,max_unsec_exposure_inPct,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,35,15,11,29,29,0,0,0,...,0.0,0.0,0.0,13.333,1,0,PL,PL,696,P2
1,2,47,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,0.860,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,11,3,9,25,25,1,9,8,...,0.0,0.0,0.0,5741.667,1,0,ConsumerLoan,others,693,P2
3,4,-99999,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,9.900,0,0,others,others,673,P2
4,5,583,-99999,-99999,0,-99999,0,0,0,0,...,0.0,0.0,0.0,-99999.000,0,0,AL,AL,753,P1


In [105]:
# these columns have -99999 values more than 10,000 times
columns_to_be_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [106]:
# axis = 1 shows is for columns
df2 = df2.drop(columns_to_be_removed,axis = 1)

In [107]:
df2.shape

(51336, 54)

In [108]:
for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

In [109]:
df2.shape

(42066, 54)

In [110]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

In [111]:
df2.isna().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

In [112]:
# Checking common column names
for i in list(df1.columns):
    if i in list(df2.columns):
        print(i)

PROSPECTID


In [113]:
# Merging two dataframes, df1 and df2, using the common column "PROSPECTID"
# The 'how' parameter specifies the type of merge to perform, which is 'inner' in this case.
# An 'inner' merge keeps only the rows with matching "PROSPECTID" in both dataframes.

df = pd.merge(df1, df2, how='inner', left_on=["PROSPECTID"], right_on=['PROSPECTID']) 

# - 'df': The resulting dataframe after merging.
# - 'df1' and 'df2': The two dataframes being merged.
# - 'how="inner"': This type of merge only includes rows with matching values in "PROSPECTID" from both dataframes.
# - 'left_on' and 'right_on': These specify the column to match on, which is "PROSPECTID" in both dataframes.


In [114]:
df

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.00,0.167,0.833,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42059,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
42060,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
42061,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
42062,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [115]:
df.shape

(42064, 79)

In [116]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [117]:
df.isna().sum()

PROSPECTID             0
Total_TL               0
Tot_Closed_TL          0
Tot_Active_TL          0
Total_TL_opened_L6M    0
                      ..
GL_Flag                0
last_prod_enq2         0
first_prod_enq2        0
Credit_Score           0
Approved_Flag          0
Length: 79, dtype: int64

In [118]:
# Check how many columns are categorical
for i in df.columns:
    if df[i].dtype == 'object':
        print(i) 

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [119]:
# Loop through each variable in the specified list
for i in ["MARITALSTATUS", "EDUCATION", "GENDER", "last_prod_enq2", "first_prod_enq2"]:
    # Calculate the Chi-Square statistic and p-value for the current variable 'i'
    # using a contingency table created from the dataframe 'df'
    # The contingency table compares the categories of the variable 'i' with 'Approved_Flag'

    # In Python, the underscore _ is often used as a placeholder for values that are not needed or used in the code
    # the chi2_contingency function returns four values:

    # chi2: The Chi-Square statistic.
    # pval: The p-value of the test.
    # _: The degrees of freedom (not used in this case, hence the underscore).
    # _: The expected frequencies table (also not used here, hence another underscore).
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df["Approved_Flag"]))
    
    # Print the variable name and the corresponding p-value from the Chi-Square test
    print(i, "----", pval)


MARITALSTATUS ---- 3.578180861038862e-233
EDUCATION ---- 2.6942265249737532e-30
GENDER ---- 1.907936100186563e-05
last_prod_enq2 ---- 0.0
first_prod_enq2 ---- 7.84997610555419e-287


In [120]:
numeric_columns = []
for i in df.columns:
    # Check if the data type of the column is not 'object' (i.e., it's a numeric type)
    # and also ensure that the column name is not in the list of excluded columns
    if df[i].dtype != "object" and i not in ["PROSPECTID","Approved_Flag"]:
        numeric_columns.append(i)

In [121]:
numeric_columns

['Total_TL',
 'Tot_Closed_TL',
 'Tot_Active_TL',
 'Total_TL_opened_L6M',
 'Tot_TL_closed_L6M',
 'pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'pct_active_tl',
 'pct_closed_tl',
 'Total_TL_opened_L12M',
 'Tot_TL_closed_L12M',
 'pct_tl_open_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'Auto_TL',
 'CC_TL',
 'Consumer_TL',
 'Gold_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'num_times_delinquent',
 'max_recent_level_of_deliq',
 'num_deliq_6mts',
 'num_deliq_12mts',
 'num_deliq_6_12mts',
 'num_times_30p_dpd',
 'num_times_60p_dpd',
 'num_std',
 'num_std_6mts',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_6mts',
 'num_dbt_12mts',
 'num_lss',
 'num_lss_6mts',
 'num_lss_12mts',
 'recent_level_of_deliq',
 'tot_enq',
 'CC_enq',
 'CC_enq_L6m',
 'CC_enq_L12m',
 'PL_enq',
 'PL_enq_L6m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L12m',
 'enq_L6m',
 'enq_L3m',

In [122]:
# Create a new DataFrame containing only the numeric columns identified earlier
vif_data = df[numeric_columns]

# Get the total number of numeric columns
total_columns = vif_data.shape[1]

# Initialize an empty list to keep track of the columns that will be kept
columns_to_be_kept = []

# Initialize the column index to start from 0
column_index = 0

# Loop through each index from 0 to the total number of numeric columns
for i in range(0, total_columns):
    # Calculate the Variance Inflation Factor (VIF) for the column at the current index
    # 'variance_inflation_factor' function requires the DataFrame and the index of the column to calculate VIF
    vif_value = variance_inflation_factor(vif_data, column_index)
    
    # Print the index of the column and its VIF value
    print(column_index, "---", vif_value)

    # Check if the VIF value is within an acceptable range
    if vif_value <= 6:
        # If the VIF is acceptable, add the column name to the list of columns to be kept
        columns_to_be_kept.append(numeric_columns[i])
        # Increment the column index to move to the next column
        column_index = column_index + 1
    else:
        # If the VIF is too high, drop the column from the DataFrame
        # The column is removed from 'vif_data' using its name
        vif_data = vif_data.drop([numeric_columns[i]], axis=1)


c:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


c:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000327
0 --- 6.520647877790933
0 --- 5.149501618212625
1 --- 2.611111040579735


c:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.0996533816467355
3 --- 5.581352009642762
4 --- 1.9855843530987785


c:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299253
9 --- 4.377876915347322
10 --- 2.2078535836958433
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


c:\Users\Gaurav\OneDrive\Desktop\Projects\Machine Learning Projects\Credit Risk Modeling (ML)\env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427232
15 --- 1.421005001517573
16 --- 8.083255010190323
16 --- 1.6241227524040112
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824577
20 --- 2.623397553527229
21 --- 2.2959970812106167
22 --- 7.360578319196446
22 --- 2.1602387773102554
23 --- 2.8686288267891458
24 --- 6.458218003637277
24 --- 2.8474118865638265
25 --- 4.7531981562840855
26 --- 16.227354755948223
26 --- 6.424377256363877
26 --- 8.887080381808687
26 --- 2.3804746142952653
27 --- 8.60951347651454
27 --- 13.06755093547673
27 --- 3.5000400566546555
28 --- 1.908795587481377
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512924
31 --- 3.424171203217697
32 --- 10.175021454450935
32 --- 6.408710354561301
32 --- 1.001151196262561
33 --- 3.069197305397274
34 --- 2.8091261600643724
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.833164974053

In [123]:
print(len(columns_to_be_kept))

39


In [130]:
# Check Anova for columns_to_be_kept
# Import the f_oneway function from scipy.stats for performing ANOVA
from scipy.stats import f_oneway

# Initialize an empty list to store the names of columns that will be kept after ANOVA testing
columns_to_be_kept_numerical = []

# Loop through each column name in the list 'columns_to_be_kept'
for i in columns_to_be_kept:
    # Convert the column values and 'Approved_Flag' values into lists
    a = list(df[i])
    b = list(df["Approved_Flag"])

    # Separate the data into groups based on 'Approved_Flag' values
    group_p1 = [value for value, group in zip(a, b) if group == 'P1']
    group_p2 = [value for value, group in zip(a, b) if group == 'P2']
    group_p3 = [value for value, group in zip(a, b) if group == 'P3']
    group_p4 = [value for value, group in zip(a, b) if group == 'P4']

    # Perform ANOVA test across the four groups
    f_statistics, p_value = f_oneway(group_p1, group_p2, group_p3, group_p4)

    # If the p-value is less than or equal to 0.05, it indicates a significant difference between groups
    # Therefore, the column is added to 'columns_to_be_kept_numerical'
    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)


In [131]:
print(len(columns_to_be_kept_numerical))

37


In [132]:
# Assigning numerical values as per the education level
df.loc[df['EDUCATION'] == "SSC", ["EDUCATION"]]                 = 1
df.loc[df['EDUCATION'] == "12TH", ["EDUCATION"]]                = 2
df.loc[df['EDUCATION'] == "GRADUATE", ["EDUCATION"]]            = 3
df.loc[df['EDUCATION'] == "UNDER GRADUATE", ["EDUCATION"]]      = 3
df.loc[df['EDUCATION'] == "POST-GRADUATE", ["EDUCATION"]]       = 4
df.loc[df['EDUCATION'] == "OTHERS", ["EDUCATION"]]              = 1
df.loc[df['EDUCATION'] == "PROFESSIONAL", ["EDUCATION"]]        = 3

In [133]:
# his line displays the count of unique values in the EDUCATION column of the DataFrame df. 
# It helps to understand the distribution of different education levels in the dataset.
df['EDUCATION'].value_counts()

# This line converts the values in the EDUCATION column to integers
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [136]:
# Import the pandas library
import pandas as pd

# Assume df is your original DataFrame with categorical columns

# Apply one-hot encoding to the specified categorical columns in df
# The function pd.get_dummies() converts categorical data into a set of binary columns
# Each unique value in the specified columns will become a separate column in the new DataFrame df_encoded
# The original categorical columns 'MARITALSTATUS', 'GENDER', 'last_prod_enq2', and 'first_prod_enq2' 
# will be replaced by the new binary columns
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'])

# df_encoded is now a new DataFrame containing all the original columns from df except the specified 
# categorical columns, which have been replaced by their one-hot encoded binary columns
# Each original categorical value is represented by a separate column with binary values (0 or 1)


In [137]:
df_encoded.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,False,False,True,False,False,False,False,False,True,False
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,True,False,False,False,False,False,True,False,False,False
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,True,False,False,False,False,False,False,False,False,True
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,False,False,False,False,True,False,False,False,False,False
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,True,False,False,False,False,False,False,False,True,False


In [138]:
df_encoded.describe()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,Credit_Score
count,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.00000,42064.000000,42064.000000,42064.000000,42064.000000,...,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000,42064.000000
mean,25649.827477,5.262980,2.967383,2.295597,0.812643,0.48992,0.179032,0.097783,0.577452,0.422548,...,0.883693,0.102962,0.193063,0.219169,0.074833,0.195497,0.064186,0.252235,0.056580,679.326336
std,14844.173396,7.463383,6.141098,2.404086,1.383559,1.05892,0.278043,0.210957,0.366110,0.366110,...,40.622275,0.303913,0.394707,0.395100,0.250658,0.367414,0.225989,0.434300,0.231042,21.133619
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,469.000000
25%,12776.750000,1.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.286000,0.000000,...,0.152000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,668.000000
50%,25706.500000,3.000000,1.000000,2.000000,0.000000,0.00000,0.000000,0.000000,0.545000,0.455000,...,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,679.000000
75%,38518.250000,6.000000,3.000000,3.000000,1.000000,1.00000,0.333000,0.100000,1.000000,0.714000,...,0.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,690.000000
max,51336.000000,235.000000,216.000000,47.000000,27.000000,19.00000,1.000000,1.000000,1.000000,1.000000,...,6327.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,809.000000


In [139]:
df_encoded["Approved_Flag"]

0        P2
1        P2
2        P2
3        P1
4        P3
         ..
42059    P4
42060    P1
42061    P3
42062    P2
42063    P2
Name: Approved_Flag, Length: 42064, dtype: object

**Machine Learning Model Fitting Using Random Forest**

In [142]:
y = df_encoded["Approved_Flag"]
x = df_encoded.drop(["Approved_Flag"], axis=1)

In [143]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42) 

In [144]:

# Initialize the Random Forest classifier
# n_estimators=200 sets the number of trees in the forest to 200
# random_state=42 ensures reproducibility by setting the seed for random number generation
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)

In [145]:
# Training
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [146]:
y_pred = rf_classifier.predict(x_test)

In [147]:
# Checking accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print()
print(f"Accuracy: {accuracy}")
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)



Accuracy: 0.9900154522762391


In [150]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class: {v}")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Class: p1
Precision : 0.9465290806754222
Recall : 0.995069033530572
F1 Score: 0.9701923076923077

Class: p2
Precision : 0.9954617205998422
Recall : 1.0
F1 Score: 0.9977256995945812

Class: p3
Precision : 0.9968102073365231
Recall : 0.9433962264150944
F1 Score: 0.9693679720822024

Class: p4
Precision : 1.0
Recall : 0.9961127308066083
F1 Score: 0.9980525803310614



**Fitting Model Using XGBoost**

In [155]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


# Initialize the XGBoost classifier
# objective='multi:softmax' sets the objective function for multiclass classification, where the model outputs the class with the highest probability
# num_class=4 specifies that there are 4 distinct classes in the target variable
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax', num_class=4)


In [156]:
# Extract the target variable (labels) from the DataFrame
# 'y' will contain the values from the 'Approved_Flag' column
y = df_encoded["Approved_Flag"]

# Extract the features (input variables) from the DataFrame
# 'x' will contain all columns from df_encoded except for 'Approved_Flag'
# axis=1 indicates that the operation is column-wise
x = df_encoded.drop(["Approved_Flag"], axis=1)


In [154]:
Label_encoder = LabelEncoder()
y_encoded = Label_encoder.fit_transform(y)

In [157]:
# Initialize the LabelEncoder
Label_encoder = LabelEncoder()

# Fit the LabelEncoder on the target variable 'y' and transform it into numeric form
# 'y_encoded' will contain the encoded labels, where each unique category in 'y' is assigned a unique integer value
y_encoded = Label_encoder.fit_transform(y)


In [168]:
x_train, x_test, y_train, y_test = train_test_split(x,y_encoded, test_size=0.2, random_state=42)    

In [182]:
xgb_classifier.fit(x_train, y_train)
y_pred1 = xgb_classifier.predict(x_test)

In [183]:
accuracy = accuracy_score(y_test, y_pred1)
print()
print(f"Accuracy : {accuracy:.2f}")
print()



Accuracy : 1.00



In [184]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred1)


In [185]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class: {v}")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class: p1
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p2
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p3
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p4
Precision : 1.0
Recall : 1.0
F1 Score: 1.0



**Decision Tree**

In [175]:
from sklearn.tree import DecisionTreeClassifier

# Extract the target variable (labels) from the DataFrame
# 'y' will contain the values from the 'Approved_Flag' column
y = df_encoded["Approved_Flag"]

# Extract the features (input variables) from the DataFrame
# 'x' will contain all columns from df_encoded except for 'Approved_Flag'
# axis=1 indicates that the operation is column-wise
x = df_encoded.drop(["Approved_Flag"], axis=1)



In [176]:
x_train, x_test, y_train, y_test = train_test_split(x,y_encoded, test_size=0.2, random_state=42)    

In [186]:
from sklearn.tree import DecisionTreeClassifier

# Initialize the Decision Tree classifier with specified hyperparameters
# max_depth=20 sets the maximum depth of the tree to 20 levels, controlling the model's complexity
# min_samples_split=10 requires at least 10 samples to split an internal node, preventing the model from learning overly specific patterns
dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)

# Fit the Decision Tree model to the training data
# 'x_train' contains the feature variables for training
# 'y_train' contains the target variable for training
dt_model.fit(x_train, y_train)

# Predict the target variable for the test data
# 'x_test' contains the feature variables for testing
# 'y_pred' will contain the predicted labels for the test set based on the fitted model
y_pred2 = dt_model.predict(x_test)


In [187]:
accuracy = accuracy_score(y_test, y_pred2)
print()
print(f"Accuracy : {accuracy:.2f}")
print()



Accuracy : 1.00



In [189]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred2)


In [190]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class: {v}")
    print(f"Precision : {precision[i]}")
    print(f"Recall : {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()

Class: p1
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p2
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p3
Precision : 1.0
Recall : 1.0
F1 Score: 1.0

Class: p4
Precision : 1.0
Recall : 1.0
F1 Score: 1.0



**HYPERPARAMETER TUNING**

In [193]:
# Define the hyperparameter grid for tuning the XGBoost model
# This dictionary contains different values for each hyperparameter to test
param_grid = {
    "colsample_bytree" : [0.1, 0.3, 0.5, 0.7, 0.9],  # Fraction of features to be randomly sampled for each tree
    "learning_rate"    : [0.001, 0.01, 0.1, 1],      # Step size shrinkage used to prevent overfitting
    "max_depth"        : [3, 5, 8, 10 ],              # Maximum depth of the trees
    "alpha"            : [1, 10, 100],                # L1 regularization term on weights
    "n_estimators"     : [10, 50, 100]                # Number of boosting rounds (trees)
}

# Initialize an index variable for tracking the combination of hyperparameters tested
index = 0

# Create a dictionary to store the results of hyperparameter tuning
# This will hold information about each tested combination and its performance
answer_grid = {
    "combination"       : [],      # List to store the combination of hyperparameters used
    "train_Accuracy"    : [],      # List to store training accuracy for each combination
    "test_Accuracy"     : [],      # List to store test accuracy for each combination
    "colsample_bytree"  : [],      # List to store the 'colsample_bytree' value for each combination
    "learning_rate"     : [],      # List to store the 'learning_rate' value for each combination
    "max_depth"         : [],      # List to store the 'max_depth' value for each combination
    "alpha"             : [],      # List to store the 'alpha' value for each combination
    "n_estimators"      : []       # List to store the 'n_estimators' value for each combination
}


In [201]:
# Initialize the answer_grid dictionary
answer_grid = {
    "combination": [],
    "train_accuracy": [],
    "test_accuracy": [],
    "colsample_bytree": [],
    "learning_rate": [],
    "max_depth": [],
    "alpha": [],
    "n_estimators": []
}

# Loop through each combination of hyperparameters
for colsample_bytree in param_grid["colsample_bytree"]:
    for learning_rate in param_grid["learning_rate"]:
        for max_depth in param_grid["max_depth"]:
            for alpha in param_grid["alpha"]:
                for n_estimators in param_grid["n_estimators"]:

                    index += 1

                    # Define and train the XGBoost model
                    model = xgb.XGBClassifier(objective="multi:softmax",
                                              num_class=4,
                                              colsample_bytree=colsample_bytree,
                                              learning_rate=learning_rate,
                                              max_depth=max_depth,
                                              alpha=alpha,
                                              n_estimators=n_estimators)

                    # Define target and features
                    y = df_encoded["Approved_Flag"]
                    x = df_encoded.drop(["Approved_Flag"], axis=1)

                    # Encode the target variable to integer labels
                    label_encoder = LabelEncoder()
                    y_encoded = label_encoder.fit_transform(y)

                    # Split the data into training and testing sets
                    x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

                    # Fit the model
                    model.fit(x_train, y_train)

                    # Predict on training and testing sets
                    y_pred_train = model.predict(x_train)
                    y_pred_test = model.predict(x_test)

                    # Calculate train and test accuracies
                    train_accuracy = accuracy_score(y_train, y_pred_train)
                    test_accuracy = accuracy_score(y_test, y_pred_test)

                    # Append the results to the dictionary
                    answer_grid["combination"].append(index)
                    answer_grid["train_accuracy"].append(train_accuracy)
                    answer_grid["test_accuracy"].append(test_accuracy)
                    answer_grid["colsample_bytree"].append(colsample_bytree)
                    answer_grid["learning_rate"].append(learning_rate)
                    answer_grid["max_depth"].append(max_depth)
                    answer_grid["alpha"].append(alpha)
                    answer_grid["n_estimators"].append(n_estimators)

                    print(f"Combinations: {index}")
                    print(f"Colsample_bytree : {colsample_bytree}, max_depth : {max_depth}, Alpha : {alpha},  n_estimators : {n_estimators}")
                    print(f"Train_accuracy: {train_accuracy}")
                    print(f"Test_accuracy : {test_accuracy}")
                    print("-" * 30)


Combinations: 8
Colsample_bytree : 0.1, max_depth : 3, Alpha : 1,  n_estimators : 10
Train_accuracy: 0.6493120561053163
Test_accuracy : 0.6409128729347439
------------------------------
Combinations: 9
Colsample_bytree : 0.1, max_depth : 3, Alpha : 1,  n_estimators : 50
Train_accuracy: 0.665210543520252
Test_accuracy : 0.658029240461191
------------------------------
Combinations: 10
Colsample_bytree : 0.1, max_depth : 3, Alpha : 1,  n_estimators : 100
Train_accuracy: 0.6598615197170961
Test_accuracy : 0.6539878759063354
------------------------------
Combinations: 11
Colsample_bytree : 0.1, max_depth : 3, Alpha : 10,  n_estimators : 10
Train_accuracy: 0.6483314017414045
Test_accuracy : 0.6390110543206942
------------------------------
Combinations: 12
Colsample_bytree : 0.1, max_depth : 3, Alpha : 10,  n_estimators : 50
Train_accuracy: 0.6636058363793053
Test_accuracy : 0.6568406038274099
------------------------------
Combinations: 13
Colsample_bytree : 0.1, max_depth : 3, Alpha : 10